## Mission Parse Playground

In [1]:
import numpy as np
import pandas as pd
import os

Geting athe data from one of the sample files

In [4]:
filedirectory = '/home/mechatronics/catkin_ws/src/blackflag/mission_parse/samples/mission_files'
filename = 'all_stations.txt'

filepath = os.path.join(filedirectory,filename)
mission = open(filepath, 'r+')
mission_data = mission.readline()
print(mission_data)

AV1 90, BV2 270, CV1 90, DA B1 U, EV3 1, FV3 1, GB B2 D, 560


Seperate missions into individual data segments

In [10]:
segments = mission_data.split(",")
segments = [segment.strip() for segment in segments]
time = segments[-1]
segments = segments[0:-1]
print(segments)
print(time)

['AV1 90', 'BV2 270', 'CV1 90', 'DA B1 U', 'EV3 1', 'FV3 1', 'GB B2 D']
560


In [12]:
feature_map = {'V1':'Gate Valve', 'V2':'Large Valve', 'V3':'Stopcock Valve', 'A':'Breaker Box', 'B':'Breaker Box'}

parsed_command = []

for segment in segments:
    command = []
    command.append(segment[0])
    feature = segment[1] if segment[1]=='A' or segment[1] == 'B' else segment[1:3]
    command.append(feature_map[feature])
    if feature=='A' or feature=='B':
        info = segment.split(" ")
        button = [0,0,0]
        info_button = 1 if info[2] == 'U' else -1
        button[int(info[1][1])] = info_button
        command.append(button)
    else:
        info = segment.split(" ")
        command.append(int(info[1]))
    parsed_command.append(command)

print(parsed_command)

[['A', 'Gate Valve', 90], ['B', 'Large Valve', 270], ['C', 'Gate Valve', 90], ['D', 'Breaker Box', [0, 1, 0]], ['E', 'Stopcock Valve', 1], ['F', 'Stopcock Valve', 1], ['G', 'Breaker Box', [0, 0, -1]]]


Create Function out of this

In [8]:
def parse_mission(filename):
    # Store mission files in the directory below
    filedirectory = '/home/mechatronics/catkin_ws/src/blackflag/mission_parse/samples/mission_files'
    
    filepath = os.path.join(filedirectory,filename)
    mission = open(filepath, 'r+')
    mission_data = mission.readline()

    segments = mission_data.split(",")
    segments = [segment.strip() for segment in segments]
    time = int(segments[-1])
    segments = segments[0:-1]

    feature_map = {'V1':'Gate Valve', 'V2':'Large Valve', 'V3':'Stopcock Valve', 'A':'Breaker Box', 'B':'Breaker Box'}

    parsed_command = []

    for segment in segments:
        command = []
        command.append(segment[0])
        feature = segment[1] if segment[1]=='A' or segment[1] == 'B' else segment[1:3]
        command.append(feature_map[feature])
        if feature=='A' or feature=='B':
            info = segment.split(" ")
            button = [0,0,0]
            info_button = 1 if info[2] == 'U' else -1
            button[int(info[1][1])-1] = info_button
            command.append(button)
        else:
            info = segment.split(" ")
            command.append(int(info[1]))
        parsed_command.append(command)
        
    parsed_command = sorted(parsed_command, key = lambda x: x[0])
    return parsed_command, time


def encode(plan):
    code = [task[0] for task in plan]
    encoded_plan = '00000000'
    ch = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for i in range(0,8):
        if ch[i] in code:
            encoded_plan = encoded_plan[:i]+'1'+encoded_plan[i+1:]
                
    return encoded_plan

In [10]:
#Test
for file in os.listdir(filedirectory):
    print(f'File Name = {file}')
    
    parsed_command, time = parse_mission(file)
    encoded = encode(parsed_command)
    for task in parsed_command:
        print(task)
    print(f'Time = {time} secs')
    print(f'Encoded Plan : {encoded}\n')

File Name = single_station.txt
['B', 'Large Valve', 0]
Time = 80 secs
Encoded Plan : 01000000

File Name = sample.txt
['B', 'Large Valve', 180]
['C', 'Gate Valve', 90]
['D', 'Breaker Box', [0, -1, 0]]
['E', 'Stopcock Valve', 0]
['F', 'Stopcock Valve', 0]
Time = 400 secs
Encoded Plan : 01111100

File Name = hard.txt
['A', 'Gate Valve', 180]
['B', 'Large Valve', 180]
['C', 'Gate Valve', 270]
['D', 'Breaker Box', [0, 0, -1]]
['D', 'Breaker Box', [0, 1, 0]]
['E', 'Stopcock Valve', 1]
['F', 'Stopcock Valve', 1]
['H', 'Breaker Box', [1, 0, 0]]
Time = 610 secs
Encoded Plan : 11111101

File Name = all_stations.txt
['A', 'Gate Valve', 90]
['B', 'Large Valve', 270]
['C', 'Gate Valve', 90]
['D', 'Breaker Box', [1, 0, 0]]
['E', 'Stopcock Valve', 1]
['F', 'Stopcock Valve', 1]
['G', 'Breaker Box', [0, -1, 0]]
Time = 560 secs
Encoded Plan : 11111110

File Name = repeat.txt
['B', 'Large Valve', 0]
['B', 'Large Valve', 180]
['E', 'Stopcock Valve', 1]
Time = 220 secs
Encoded Plan : 01001000

